In [ ]:
import Read_mseed_from_Drive as MH
import pandas as pd

In [ ]:

# Generate credentials, load from token if it exists, otherwise create a token
SCOPES = ["https://www.googleapis.com/auth/drive.readonly"]

credential_json = 'ENTER YOUR CREDENTIALS JSON FILE PATH HERE'
token_json = 'ENTER YOUR TOKEN JSON FILE PATH HERE'

creds = MH.get_credentials(credential_json,token_json,SCOPES)

# --- 1) Authenticate -------------------------------------------------
drive = MH.build("drive", "v3", credentials=creds)

# Example using the folder walk search for the MergedData folder
file_type = '.mseed'
deep_folder_id = 'ENTER YOUR DEEP FOLDER ID HERE' # This is the folder ID of your gdrive link

all_files = MH.list_all_mseed_files(drive,deep_folder_id,file_type)
session = MH.requests.Session()
session.headers.update({"Authorization": "Bearer " + creds.token})

# Put header info into dictionary; currently there is no check for bad records
# Put data from all files into on stream object
all_headers = []
streams = []
for f in all_files:
    #print(f)
    try:
        print(f"Reading {f['name']}")
        st = MH.fetch_full_mseed(f["id"],session)
        #-----------
        #### this is for fetchin the header bytes into a data frame: all_headers
        b = MH.fetch_header_bytes(f["id"])
        hdr = MH.parse_mseed_header(b)
        hdr["name"] = f["name"]
        all_headers.append(hdr)
        #-----------
        for tr in st: streams.append(tr)
    except Exception as e:
        print(e)


df = pd.DataFrame(all_headers)
